In [ ]:
import os
os.environ['HF_HOME'] = '/home/tl688/scratch/'
os.environ['HF_HUB_HOME'] = '/home/tl688/scratch/'
os.environ['HF_TOKEN'] = ''

def weighted_mse_loss(input_data, target, weight):
    return torch.sum(weight * (input_data - target) ** 2)

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/home/tl688/scratch/qwen3_1.7b_full_sft_verifier_0830/checkpoint-82/"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

In [3]:
PROMPT_TEMPLATE = """# Instruction
Read the question and choose the question type from the options below.

# Available question types
0: Reasoning-based pathology visual question answer
1: Pathology patch summarization
2: Cross-modality generation
3: Diagnosis correction

# Question
{Question}

# Answer format
Only return one integer: 0 or 1 or 2 or 3.
"""

In [6]:
question = 'Based on the histopathologic features of irregular, atypical glandular proliferation with hyperchromatic nuclei, high nuclear-to-cytoplasmic ratio, loss of normal architecture, and a desmoplastic stromal response consistent with endometrioid (type I) endometrial adenocarcinoma, which molecular genetic alteration is most likely to be identified in this tumor?'

In [7]:
# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": PROMPT_TEMPLATE.format(Question = question )}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: <think>

</think>
content: 0
